<a href="https://colab.research.google.com/github/daycardoso/bert-vs-modernbert-valueeval24/blob/main/valores_modern_bert_final1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inicialização

In [1]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import userdata
import os
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['WANDB_API_KEY'] = userdata.get('WANDB_API_KEY')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import json
import os # Adicionado para construir os caminhos

# Carregar os Dados em Ingles
home = "/content/drive/MyDrive/Mestrado/DetectionOfHumanValuesInTexts/Colab_Experimentos/"

folder_treino = home + "training-english/"
folder_validacao = home + "validation-english/"
folder_teste = home + "test-english/"

# Json value-categories - Carregue o JSON uma única vez
caminho_json_valores = os.path.join(home, "value-categories.json")
with open(caminho_json_valores, 'r') as f:
    categorias_valores = json.load(f)

# Json value-categories
categorias_valores = json.load(open(home + "value-categories.json"))


# Aplicando o pre-processamento de adição de contexto do Hierocles of Alexandria at Touché

# Treinamento dos modelos para cada direction

In [3]:
!pip install "numpy<2.0"

In [4]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [5]:
import os
import gc
import random
import numpy as np
import pandas as pd
import torch
import datasets
import wandb
from scipy.special import expit as sigmoid
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from tqdm.auto import tqdm
from transformers import (
    AutoTokenizer,
    AutoConfig,
    ModernBertForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    DataCollatorWithPadding
)

# Configuração para reduzir fragmentação de memória
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# --- 1. Inicialização e Configuração Global ---

try:
    wandb.init(
        project='touche_multi_head',
        name='modern-bert_seq_class_19_values_no_context_10epochs'
    )
except Exception as e:
    print(f"W&B initialization failed: {e}. Running without logging.")

VALORES = [
    'Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism',
    'Achievement', 'Power: dominance', 'Power: resources', 'Face',
    'Security: personal', 'Security: societal', 'Tradition', 'Conformity: rules',
    'Conformity: interpersonal', 'Humility', 'Benevolence: caring',
    'Benevolence: dependability', 'Universalism: concern', 'Universalism: nature',
    'Universalism: tolerance'
]
NUM_LABELS = len(VALORES)
ID2LABEL = {i: l for i, l in enumerate(VALORES)}
LABEL2ID = {l: i for i, l in enumerate(VALORES)}

PRETRAINED_MODEL = 'answerdotai/ModernBERT-base'
MAX_LENGTH = 512

# --- 2. Preparação do Tokenizador ---

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

# --- 3. Funções de Processamento de Dados ---

def preprocess_function(examples):
    """Tokeniza os textos do dataset."""
    return tokenizer(examples['Text'], padding='max_length', truncation=True, max_length=MAX_LENGTH)

def load_and_process_dataset(directory, tokenizer_instance):
    """Carrega, mescla, processa e tokeniza o dataset."""
    sentences_file_path = os.path.join(directory, 'final_sentences.tsv')
    labels_file_path = os.path.join(directory, 'final_labels.tsv')

    if not os.path.exists(sentences_file_path) or not os.path.exists(labels_file_path):
        raise FileNotFoundError(f"Arquivos de dataset não encontrados em {directory}")

    key_column_types = {'Text-ID': str, 'Sentence-ID': str}
    try:
        data_df = pd.read_csv(
            sentences_file_path,
            sep='\t',
            dtype=key_column_types,
            usecols=['Text-ID', 'Sentence-ID', 'Text']
        )
    except ValueError as e:
        raise ValueError(f"Verifique se 'final_sentences.tsv' contém as colunas 'Text-ID', 'Sentence-ID' e 'Text'. Erro: {e}")

    labels_df = pd.read_csv(labels_file_path, sep='\t', dtype=key_column_types)
    merged_df = pd.merge(data_df, labels_df, on=['Text-ID', 'Sentence-ID'])

    labels_matrix = merged_df[VALORES].values.astype(np.float32)
    merged_df['labels'] = [row.astype(np.float32) for row in labels_matrix]

    # Exemplo aleatório para inspeção
    random_idx = random.randint(0, len(merged_df) - 1)
    sample_info = {
        'Text-ID': merged_df['Text-ID'][random_idx],
        'Sentence-ID': merged_df['Sentence-ID'][random_idx],
        'Text': merged_df['Text'][random_idx],
        'labels': [ID2LABEL[i] for i, label in enumerate(merged_df['labels'][random_idx]) if label == 1] or 'Nenhum'
    }

    print("\n" + "="*35)
    print(f"=== Exemplo de Texto Pré-processado ({os.path.basename(directory)}) ===")
    print(f"Text-ID: {sample_info['Text-ID']}")
    print(f"Sentence-ID: {sample_info['Sentence-ID']}")
    print(f"Texto: {sample_info['Text']}")
    print(f"Rótulos: {sample_info['labels']}")
    print("="*35 + "\n")

    dataset = datasets.Dataset.from_pandas(merged_df)
    dataset = dataset.map(preprocess_function, batched=True, load_from_cache_file=False)
    valid_cols = ['input_ids', 'attention_mask', 'labels']
    dataset = dataset.remove_columns([c for c in dataset.column_names if c not in valid_cols])
    dataset.set_format("torch")
    return dataset

# --- 4. Função de Métricas ---

def compute_metrics(eval_pred):
    """Calcula métricas para avaliação multi-label."""
    logits, true_labels = eval_pred
    probs = sigmoid(logits)
    preds = (probs > 0.5).astype(int)

    f1_macro = f1_score(true_labels, preds, average='macro', zero_division=0)
    f1_micro = f1_score(true_labels, preds, average='micro', zero_division=0)
    precision_macro = precision_score(true_labels, preds, average='macro', zero_division=0)
    recall_macro = recall_score(true_labels, preds, average='macro', zero_division=0)
    subset_accuracy = (true_labels == preds).all(axis=1).mean()

    auc_scores = []
    for i in range(NUM_LABELS):
        if len(np.unique(true_labels[:, i])) > 1:
            auc_scores.append(roc_auc_score(true_labels[:, i], probs[:, i]))
    roc_auc = np.mean(auc_scores) if auc_scores else float('nan')

    return {
        'subset_accuracy': subset_accuracy,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'roc_auc': roc_auc
    }



wandb: Currently logged in as: day-cardoso (day-cardoso-ufrgs-universidade-federal-do-rio-grande-do-sul) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [6]:
# --- 5. Lógica Principal de Treinamento ---

def main():
    """Função principal que executa o pipeline."""
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f'Usando dispositivo: {device}')

    print('Carregando e processando datasets...')
    train_ds = load_and_process_dataset(folder_treino, tokenizer)
    val_ds = load_and_process_dataset(folder_validacao, tokenizer)
    test_ds = load_and_process_dataset(folder_teste, tokenizer)
    print('Datasets prontos.')

    config = AutoConfig.from_pretrained(
        PRETRAINED_MODEL,
        num_labels=NUM_LABELS,
        id2label=ID2LABEL,
        label2id=LABEL2ID,
        problem_type='multi_label_classification'
    )
    model = ModernBertForSequenceClassification.from_pretrained(PRETRAINED_MODEL, config=config)

    training_args = TrainingArguments(
        output_dir='modern-bert-seq-class-values-no-context',
        report_to='wandb',
        eval_strategy='steps',
        eval_steps=767,
        save_strategy='steps',
        save_steps=767,
        save_total_limit=2,
        learning_rate=5e-5,
        adam_epsilon=1e-8,
        num_train_epochs=20,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=4,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='f1_macro',
        greater_is_better=True,
        fp16=True if device == 'cuda' else False,
        lr_scheduler_type='linear',
        warmup_ratio=0.1,
        seed=2025,
        overwrite_output_dir=True,
        push_to_hub=True,
        hub_model_id='DayCardoso/modern-bert-seq-class-values-no-context',
    )

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
    )

    print('Iniciando treinamento...')
    torch.cuda.empty_cache()  # Liberar memória antes do treinamento
    train_res = trainer.train()

    metrics = train_res.metrics
    metrics['train_samples'] = len(train_ds)
    trainer.log_metrics('train', metrics)
    trainer.save_model()
    trainer.save_state()
    trainer.save_metrics('train', metrics)

    print('\nAvaliação final no dataset de validação...')
    torch.cuda.empty_cache()  # Liberar memória antes da avaliação
    eval_res = trainer.evaluate()
    eval_res['eval_samples'] = len(val_ds)
    trainer.log_metrics('eval', eval_res)
    trainer.save_metrics('eval', eval_res)

    print('\nAvaliação no dataset de teste...')
    torch.cuda.empty_cache()  # Liberar memória antes do teste
    test_res = trainer.evaluate(eval_dataset=test_ds, metric_key_prefix='test')
    test_res['test_samples'] = len(test_ds)
    trainer.log_metrics('test', test_res)
    trainer.save_metrics('test', test_res)
    print('Teste concluído! Métricas:', test_res)

    wandb.finish()

if __name__ == '__main__':

    main()

Usando dispositivo: cuda
Carregando e processando datasets...

=== Exemplo de Texto Pré-processado () ===
Text-ID: BG_204
Sentence-ID: 1
Texto: Unions threaten protests over 30,000 redundancies
Rótulos: ['Power: dominance']



Map:   0%|          | 0/24534 [00:00<?, ? examples/s]


=== Exemplo de Texto Pré-processado () ===
Text-ID: BG_183
Sentence-ID: 11
Texto: During this time the state of the BDZ was also not good. After the arrival of the third GERB cabinet he was replaced. Before that - during the cabinet of Plamen Oresharski, which was supported by BSP and DPS, Georgi Drumev was the manager of BDZ Freight in the period 2013-2014.
Rótulos: ['Achievement']



Map:   0%|          | 0/8099 [00:00<?, ? examples/s]


=== Exemplo de Texto Pré-processado () ===
Text-ID: DE_M_017
Sentence-ID: 35
Texto: We Free Democrats want to further develop the tried and tested framework of the World Trade Organization (WTO).
Rótulos: ['Stimulation']



...

***** train metrics *****
  epoch                    =     6.5002
  total_flos               = 50616546GF
  train_loss               =     0.3667
  train_runtime            = 1:09:47.34
  train_samples            =      24534
  train_samples_per_second =    117.182
  train_steps_per_second   =      7.327

Avaliação final no dataset de validação...


***** eval metrics *****
  epoch                   =     6.5002
  eval_f1_macro           =      0.333
  eval_f1_micro           =     0.4112
  eval_loss               =     0.2242
  eval_precision_macro    =     0.3961
  eval_recall_macro       =     0.3055
  eval_roc_auc            =     0.8269
  eval_runtime            = 0:01:10.52
  eval_samples            =       8099
  eval_samples_per_second =    114.838
  eval_steps_per_second   =     28.713
  eval_subset_accuracy    =     0.2955

Avaliação no dataset de teste...


early stopping required metric_for_best_model, but did not find eval_f1_macro so early stopping is disabled


***** test metrics *****
  epoch                   =     6.5002
  test_f1_macro           =     0.3244
  test_f1_micro           =     0.3992
  test_loss               =     0.2303
  test_precision_macro    =     0.3886
  test_recall_macro       =     0.2972
  test_roc_auc            =     0.8259
  test_runtime            = 0:01:11.87
  test_samples            =       7865
  test_samples_per_second =    109.427
  test_steps_per_second   =     27.367
  test_subset_accuracy    =      0.286
Teste concluído! Métricas: {'test_loss': 0.23032362759113312, 'test_subset_accuracy': 0.2859504132231405, 'test_f1_macro': 0.3244278970928006, 'test_f1_micro': 0.3992289036751446, 'test_precision_macro': 0.3886171345855769, 'test_recall_macro': 0.2972311171392637, 'test_roc_auc': 0.8259253780902038, 'test_runtime': 71.8743, 'test_samples_per_second': 109.427, 'test_steps_per_second': 27.367, 'epoch': 6.50016302575807, 'test_samples': 7865}


eval/f1_macro,▁▃▅▅▇▇███▇████
eval/f1_micro,▁▄▆▇▇▇████████
eval/loss,▂▁▁▁▂▁▃▃▄▅▆▇█▃
eval/precision_macro,▁▅▅█▅▆▅▃▄▄▄▄▄▃
eval/recall_macro,▁▃▄▄▆▆▇██▇██▇█
eval/roc_auc,▁▆▇█▇█▇▆▆▅▅▅▅▆
eval/runtime,█▁▁▂▂▂▃▂▂▂▁▃▂▂
eval/samples_per_second,▁██▇▇▇▆▇▇▇█▆▆▇
eval/steps_per_second,▁██▇▇▇▆▇▇▇█▆▆▇
eval/subset_accuracy,▁▃▅▆▇▇███▇█▇██
test/f1_macro,▁


In [9]:
# --- Célula para Inspeção dos Dados de Teste ---
print('\n' + '='*35)
print('=== Inspeção de Dados para Avaliação de Teste ===')
test_ds = load_and_process_dataset(folder_teste, tokenizer)
random_idx = random.randint(0, len(test_ds) - 1)
example = test_ds[random_idx]
text_decoded = tokenizer.decode(example['input_ids'], skip_special_tokens=True)
print(f"Exemplo Aleatório do Dataset de Teste:")
print(f"Text-ID: {example['input_ids'].numpy()[:10]}...")  # Mostra primeiros 10 tokens para brevidade
print(f"Texto Decodificado: {text_decoded}")
print(f"Máscara de Atenção: {example['attention_mask'].numpy()[:10]}...")  # Mostra primeiros 10 valores
print(f"Rótulos (Multi-hot): {example['labels'].numpy()}")
print(f"Rótulos Ativos: {[ID2LABEL[i] for i, label in enumerate(example['labels']) if label == 1] or 'Nenhum'}")
print('='*35 + '\n')


=== Inspeção de Dados para Avaliação de Teste ===

=== Exemplo de Texto Pré-processado () ===
Text-ID: HE_001
Sentence-ID: 31
Texto: Noble also said that the rig meets all the strictest conditions regarding environmental protection, and that all the official bodies have explicitly stated that there is no danger whatsoever from the activity.
Rótulos: ['Security: societal', 'Conformity: rules']



Map:   0%|          | 0/7865 [00:00<?, ? examples/s]

Exemplo Aleatório do Dataset de Teste:
Text-ID: [50281     3  7542   253  1388   359  3053   776  5871    13]...
Texto Decodificado: "Since the day we started our operations, we have increased our local human resources in Algeria by 10 times, and we will continue to increase this ratio through our collaborations with universities.
Máscara de Atenção: [1 1 1 1 1 1 1 1 1 1]...
Rótulos (Multi-hot): [0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
Rótulos Ativos: ['Self-direction: action', 'Achievement', 'Benevolence: caring']



In [ ]:
# prompt: realize a analise da celula anterior para ver se ha ou não a contaminação de dados

# A análise da célula anterior não revela contaminação de dados.
# O código parece bem estruturado para carregar, processar e treinar um modelo de classificação multi-label.

# Pontos relevantes da célula anterior para a análise:
# 1. **Carregamento de Dados Separado:** Os datasets de treino, validação e teste são carregados de diretórios distintos (`folder_treino`, `folder_validacao`, `folder_teste`).
# 2. **Separação Clara de Arquivos:** Dentro de cada diretório, os arquivos `final_sentences.tsv` (textos) e `final_labels.tsv` (rótulos) são carregados separadamente antes de serem mesclados. Isso garante que os rótulos correspondam corretamente aos textos dentro de cada partição do dataset.
# 3. **Criação do Dataset Hugging Face:** O `merged_df` (que contém textos e seus rótulos correspondentes) é convertido em um `datasets.Dataset`.
# 4. **Tokenização:** A função `preprocess_function` tokeniza os textos. A tokenização é aplicada a cada partição (treino, validação, teste) de forma independente.
# 5. **Remoção de Colunas:** Colunas desnecessárias são removidas após a tokenização, mantendo apenas `input_ids`, `attention_mask` e `labels`.
# 6. **Separação na Avaliação:** O treinamento e a avaliação são realizados em datasets separados (`train_ds`, `val_ds`). A avaliação final no dataset de teste (`test_ds`) também é explícita e separada.
# 7. **Exemplo de Teste:** O código ao final da célula mostra um exemplo aleatório do `test_ds`, demonstrando que os dados de teste foram carregados e processados corretamente, separados dos dados de treino/validação usados durante o treinamento.

# Em resumo, a metodologia de carregamento e processamento dos dados, utilizando diretórios e arquivos distintos para treino, validação e teste, e aplicando as transformações (mesclagem, tokenização) de forma consistente e separada para cada partição, é o procedimento padrão para evitar contaminação de dados. Não há indícios de que dados de treino ou validação estejam "vazando" ou sendo misturados ao dataset de teste, ou vice-versa, antes da avaliação final.

# Portanto, a análise da célula anterior indica que **não há contaminação de dados** no pipeline de preparação e carregamento dos datasets para treino, validação e teste.

# Avaliar nos dados de teste

In [7]:
# # prompt: Avaliar o modelo hub_model_id='DayCardoso/modernbert-base-multi-head-values-context', nos dados de teste


# # Inicializa o Trainer
# from transformers import DataCollatorWithPadding

# # O DataCollator cuida de agrupar e preencher os dados de forma inteligente
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# # --- 6. Avaliação no Dataset de Teste ---

# def evaluate_on_test(hub_model_id, test_directory, tokenizer_instance):
#     """
#     Carrega o modelo treinado do Hugging Face Hub,
#     carrega e processa o dataset de teste, e realiza a avaliação.
#     """
#     print(f'\nCarregando modelo do Hub: {hub_model_id}')
#     try:
#         # Carrega a configuração e o modelo do Hub
#         config = AutoConfig.from_pretrained(hub_model_id)
#         model = ModernBertForSequenceClassification.from_pretrained(hub_model_id, config=config)
#         print('Modelo carregado com sucesso.')
#     except Exception as e:
#         print(f"Erro ao carregar modelo do Hub {hub_model_id}: {e}")
#         return

#     print(f'Carregando e processando dataset de teste de {test_directory}...')
#     try:
#         test_ds = load_and_process_dataset(test_directory, tokenizer_instance)
#         print('Dataset de teste pronto.')
#     except FileNotFoundError as e:
#         print(f"Erro ao carregar dataset de teste: {e}")
#         return

#     # Re-configura o Trainer para avaliação no dataset de teste
#     # Usamos um TrainingArguments simples, focando apenas na avaliação
#     eval_args = TrainingArguments(
#         output_dir='test_evaluation', # Diretorio temporario para resultados da avaliacao
#         report_to='wandb', # Ainda reportamos para wandb
#         per_device_eval_batch_size=8, # Batch size pode ser maior para avaliacao
#         dataloader_num_workers=os.cpu_count() // 2, # Otimiza carregamento de dados se possivel
#     )

#     # O DataCollator é necessário para o Trainer, mesmo apenas para avaliação
#     data_collator = DataCollatorWithPadding(tokenizer=tokenizer_instance)

#     # Inicializa o Trainer com o modelo carregado e o dataset de teste
#     test_trainer = Trainer(
#         model=model,
#         args=eval_args,
#         eval_dataset=test_ds,
#         compute_metrics=compute_metrics, # Usa a mesma função de métricas
#         data_collator=data_collator,
#     )

#     print('\nIniciando avaliação no dataset de teste...')
#     test_eval_res = test_trainer.evaluate()

#     # Loga as métricas de teste para W&B e imprime
#     test_eval_res['eval_samples'] = len(test_ds)
#     print('--- Métricas de Avaliação no Teste ---')
#     for metric, value in test_eval_res.items():
#         print(f'{metric}: {value:.4f}')
#     print('---------------------------------------')

#     # Se o wandb estiver ativo, loga as métricas
#     try:
#         if wandb.run is not None:
#              wandb.log({"test_evaluation_metrics": test_eval_res})
#     except Exception as e:
#         print(f"Erro ao logar métricas de teste no W&B: {e}")


# # --- Chamada para Avaliar no Teste ---
# if __name__ == '__main__':
#     # main() # Comente ou remova a chamada ao main() de treinamento se quiser apenas avaliar
#     hub_model_id = 'DayCardoso/modernbert-base-multi-head-values-context'
#     print(f"\n{'='*50}\nIniciando avaliação no dataset de teste...\n{'='*50}")
#     evaluate_on_test(hub_model_id, folder_teste, tokenizer)
#     print(f"\n{'='*50}\nAvaliação no dataset de teste concluída.\n{'='*50}")
